**Project 3 - Part 1 (Core)**

For this assignment, you will be starting a new portfolio project that you will continue to build on over the remaining weeks of this course.

* For Part 1, you will be downloading and preparing a crime dataset for time series analysis.
* You will then answer the provided stakeholder questions and support your answers with visualizations.

**Chicago Crime Data**

* We have prepared a zip file with the Chicago crime data which you can download [here](https://drive.google.com/file/d/1avxUlCAros-R9GF6SKXqM_GopzO7VwA5/view?usp=drive_link).

* Original Source: Chicago Data Portal: Crimes 2001 to Present
    * Data Description:
        * All Crimes that were reported in the city of Chicago and their details [View Preview](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/data)

